<a href="https://colab.research.google.com/github/datadigger01/AI-Trade/blob/main/public_data_api_trade_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pprint
import json

### xml을 DataFrame으로 변환하기 ###
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

In [ ]:
#Servicekey = ""##################################################################""
#url = "http://apis.data.go.kr/1220000/nitemtrade/getNitemtradeList?strtYymm=202501&endYymm=202512&hsSgn=1902301010&numOfRows=1000&serviceKey=Ta%2BGkEF0ssmW1hDhEqXm5Xe2TWsM3pPycVRDIX9ejyt1GdQ%2FG8TkNhBRwJvnpHggECz6Kx5KBddps9oh9qjcPQ%3D%3D"

Servicekey= "##################################################################"
url = "http://apis.data.go.kr/1220000/nitemtrade/getNitemtradeList"

all_trade_data = []

for year in range(2015, 2025):
    strtYymm = f"{year}01"
    endYymm = f"{year}12"

    params = {
        'serviceKey': Servicekey,
        'strtYymm': strtYymm,
        'endYymm': endYymm,
        'hsSgn': '1902301010',
        #'numOfRows': '1000' # Added numOfRows as it is good practice to specify
    }

    response = requests.get(url, params=params)

    # 응답 상태 코드 확인
    print(f"Status Code for {year}: {response.status_code}")
    # 응답 본문 출력
    # print(f"Response Content for {year}:\n{response.text}") # Commenting out to reduce output

    contents = response.text
    xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
    rows = xml_obj.find_all('item') # Using find_all as suggested by DeprecationWarning

    # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
    row_list = [] # 행값
    name_list = [] # 열이름값
    value_list = [] #데이터값

    # xml 안의 데이터 수집
    for i in range(0, len(rows)):
        columns = rows[i].find_all()
        #첫째 행 데이터 수집
        for j in range(0,len(columns)):
            if i ==0:
                # 컬럼 이름 값 저장
                name_list.append(columns[j].name)
            # 컬럼의 각 데이터 값 저장
            value_list.append(columns[j].text)
        # 각 행의 value값 전체 저장
        row_list.append(value_list)
        # 데이터 리스트 값 초기화
        value_list=[]

    #xml값 DataFrame으로 만들기
    if row_list: # Only create DataFrame if there are rows
      trade_df = pd.DataFrame(row_list, columns=name_list)
      all_trade_data.append(trade_df)
    else:
      print(f"No data for year {year}")

# Concatenate all DataFrames
if all_trade_data:
    full_trade_df = pd.concat(all_trade_data, ignore_index=True)
    print("Full DataFrame created:")
    display(full_trade_df)
else:
    print("No data was retrieved for any year.")

Status Code for 2015: 200
Status Code for 2016: 200
Status Code for 2017: 200
Status Code for 2018: 200
Status Code for 2019: 200
Status Code for 2020: 200
Status Code for 2021: 200
Status Code for 2022: 200
Status Code for 2023: 200
Status Code for 2024: 200
Full DataFrame created:


,balPayments,expDlr,expWgt,hsCd,impDlr,impWgt,statCd,statCdCntnKor1,statKor,year
0,217335838,218799463,55377988,-,1463625,530355,-,-,-,총계
1,101666,101666,18149,1902301010,0,0,AE,아랍에미리트 연합,라면,2015.01
2,118927,118927,30850,1902301010,0,0,AO,앙골라,라면,2015.01
3,137306,137306,25485,1902301010,0,0,AS,아메리칸 사모아,라면,2015.01
4,917813,917845,205793,1902301010,32,2,AU,호주,라면,2015.01
...,...,...,...,...,...,...,...,...,...,...
10038,9785,9785,6553,1902301010,0,0,UY,우루과이,라면,2024.12
10039,422570,422570,102735,1902301010,0,0,UZ,우즈베키스탄,라면,2024.12
10040,1479603,1770343,479201,1902301010,290740,33860,VN,베트남,라면,2024.12
10041,128435,128435,26050,1902301010,0,0,WS,사모아,라면,2024.12


In [ ]:
### 먼저 full_trade_df 데이터셋 컬럼중에 balPayments, expDlr, expWgt, impWgt 컬럼에 대해서는 정수형 타입인 integer 타입으로 변환을 시키고, 두번째로 year 컬럼에 대해서는 date 타입으로 변환을 하되, 앞에서 4자리인 년도 정보만 추출되게 변경해줘.

# Convert specified columns to numeric types, coercing errors
cols = ['balPayments', 'expDlr', 'expWgt','impDlr','impWgt']
full_trade_df[cols] = full_trade_df[cols].apply(pd.to_numeric, errors='coerce').astype('Int64') # Use Int64 to allow for NaN values after coercion

# Remove rows where 'year' is '총계' as it's not a valid year
full_trade_df = full_trade_df[full_trade_df['year'] != '총계'].copy()

# Convert 'year' column to datetime, coercing errors, and then extract the year as an integer
full_trade_df['year'] = pd.to_datetime(full_trade_df['year'], errors='coerce')
full_trade_df['year'] = full_trade_df['year'].dt.year

full_trade_df.info()
display(full_trade_df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 10033 entries, 1 to 10042
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   balPayments     10033 non-null  Int64 
 1   expDlr          10033 non-null  Int64 
 2   expWgt          10033 non-null  Int64 
 3   hsCd            10033 non-null  object
 4   impDlr          10033 non-null  Int64 
 5   impWgt          10033 non-null  Int64 
 6   statCd          10033 non-null  object
 7   statCdCntnKor1  10033 non-null  object
 8   statKor         10033 non-null  object
 9   year            10033 non-null  int32 
dtypes: Int64(5), int32(1), object(4)
memory usage: 872.0+ KB


/tmp/ipython-input-3958128622.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  full_trade_df['year'] = pd.to_datetime(full_trade_df['year'], errors='coerce')


,balPayments,expDlr,expWgt,hsCd,impDlr,impWgt,statCd,statCdCntnKor1,statKor,year
1,101666,101666,18149,1902301010,0,0,AE,아랍에미리트 연합,라면,2015
2,118927,118927,30850,1902301010,0,0,AO,앙골라,라면,2015
3,137306,137306,25485,1902301010,0,0,AS,아메리칸 사모아,라면,2015
4,917813,917845,205793,1902301010,32,2,AU,호주,라면,2015
5,28059,28059,8471,1902301010,0,0,BD,방글라데시,라면,2015


In [ ]:
full_trade_df

,balPayments,expDlr,expWgt,hsCd,impDlr,impWgt,statCd,statCdCntnKor1,statKor,year
1,101666,101666,18149,1902301010,0,0,AE,아랍에미리트 연합,라면,2015
2,118927,118927,30850,1902301010,0,0,AO,앙골라,라면,2015
3,137306,137306,25485,1902301010,0,0,AS,아메리칸 사모아,라면,2015
4,917813,917845,205793,1902301010,32,2,AU,호주,라면,2015
5,28059,28059,8471,1902301010,0,0,BD,방글라데시,라면,2015
...,...,...,...,...,...,...,...,...,...,...
10038,9785,9785,6553,1902301010,0,0,UY,우루과이,라면,2024
10039,422570,422570,102735,1902301010,0,0,UZ,우즈베키스탄,라면,2024
10040,1479603,1770343,479201,1902301010,290740,33860,VN,베트남,라면,2024
10041,128435,128435,26050,1902301010,0,0,WS,사모아,라면,2024


In [ ]:
## statCd, statCdCntnKor, statKor, year 포함하여 4개의 컬럼을 기준으로 balPayments, expDlr, expWgt, impDlr, impWgt 컬럼값의 합계 즉 sum을 해주는 코드를 제공해주세요.

aggregated_df = full_trade_df.groupby(['statCd', 'statCdCntnKor1', 'statKor', 'year']).agg({
    'balPayments': 'sum',
    'expDlr': 'sum',
    'expWgt': 'sum',
    'impDlr': 'sum',
    'impWgt': 'sum'
}).reset_index() # Reset index to make columns from groupby available

display(aggregated_df)

,statCd,statCdCntnKor1,statKor,year,balPayments,expDlr,expWgt,impDlr,impWgt
0,AE,아랍에미리트 연합,라면,2015,1470020,1470020,275212,0,0
1,AE,아랍에미리트 연합,라면,2016,2478828,2478833,477032,5,0
2,AE,아랍에미리트 연합,라면,2017,3898236,3898236,820115,0,0
3,AE,아랍에미리트 연합,라면,2018,3459727,3459727,780673,0,0
4,AE,아랍에미리트 연합,라면,2019,3882220,3882220,926442,0,0
...,...,...,...,...,...,...,...,...,...
1325,ZM,잠비아,라면,2023,6,6,19,0,0
1326,ZW,짐바브웨,라면,2016,10,10,15,0,0
1327,ZZ,기타국,라면,2016,573,573,2000,0,0
1328,ZZ,기타국,라면,2018,-457,0,0,457,30


In [ ]:
### 해당 데이터셋에 컬럼을 하나 추가하고 싶은데, statCd별로 2019년도 expWgt 값만을 별도의 데이터셋으로 추출하고 추출된 데이터셋과 aggregated_df 데이터셋을 statCd 컬럼값을 기준으로 하여 merge를 하되, 이때의 추가된 컬럼의 이름은 'expWgt_2019'로 설정해주는 코드를 제공해주세요.


# Extract 'statCd' and 'expWgt' for the year 2019
expWgt_2019_df = aggregated_df[aggregated_df['year'] == 2019][['statCd', 'expWgt']].copy()

# Rename the 'expWgt' column to 'expWgt_2019'
expWgt_2019_df = expWgt_2019_df.rename(columns={'expWgt': 'expWgt_2019'})

# Merge the new DataFrame with the original aggregated_df on 'statCd'
merged_df = pd.merge(aggregated_df, expWgt_2019_df, on='statCd', how='left')

# Display the merged DataFrame
display(merged_df)

,statCd,statCdCntnKor1,statKor,year,balPayments,expDlr,expWgt,impDlr,impWgt,expWgt_2019_x,expWgt_2019_y
0,AE,아랍에미리트 연합,라면,2015,1470020,1470020,275212,0,0,<NA>,926442
1,AE,아랍에미리트 연합,라면,2016,2478828,2478833,477032,5,0,<NA>,926442
2,AE,아랍에미리트 연합,라면,2017,3898236,3898236,820115,0,0,<NA>,926442
3,AE,아랍에미리트 연합,라면,2018,3459727,3459727,780673,0,0,<NA>,926442
4,AE,아랍에미리트 연합,라면,2019,3882220,3882220,926442,0,0,926442,926442
...,...,...,...,...,...,...,...,...,...,...,...
1325,ZM,잠비아,라면,2023,6,6,19,0,0,<NA>,412
1326,ZW,짐바브웨,라면,2016,10,10,15,0,0,<NA>,<NA>
1327,ZZ,기타국,라면,2016,573,573,2000,0,0,<NA>,230
1328,ZZ,기타국,라면,2018,-457,0,0,457,30,<NA>,230


In [ ]:
### expWgt 컬럼값을 expWgt_2019_y 컬럼값으로 나누어준 값을 expWgt_ratio_2019 컬럼명으로 추가해주는 코드를 제공해주세요.

# Calculate the ratio of 'expWgt' to 'expWgt_2019_y' and create a new column
merged_df['expWgt_ratio_2019'] = merged_df['expWgt'] / merged_df['expWgt_2019_y']

# Display the updated DataFrame
display(merged_df)

,statCd,statCdCntnKor1,statKor,year,balPayments,expDlr,expWgt,impDlr,impWgt,expWgt_2019_x,expWgt_2019_y,expWgt_ratio_2019
0,AE,아랍에미리트 연합,라면,2015,1470020,1470020,275212,0,0,<NA>,926442,0.297063
1,AE,아랍에미리트 연합,라면,2016,2478828,2478833,477032,5,0,<NA>,926442,0.514908
2,AE,아랍에미리트 연합,라면,2017,3898236,3898236,820115,0,0,<NA>,926442,0.885231
3,AE,아랍에미리트 연합,라면,2018,3459727,3459727,780673,0,0,<NA>,926442,0.842657
4,AE,아랍에미리트 연합,라면,2019,3882220,3882220,926442,0,0,926442,926442,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1325,ZM,잠비아,라면,2023,6,6,19,0,0,<NA>,412,0.046117
1326,ZW,짐바브웨,라면,2016,10,10,15,0,0,<NA>,<NA>,<NA>
1327,ZZ,기타국,라면,2016,573,573,2000,0,0,<NA>,230,8.695652
1328,ZZ,기타국,라면,2018,-457,0,0,457,30,<NA>,230,0.0


In [ ]:
## expWgt_ratio_2019 컬럼값이 missing value 인 경우 제거를 한후 statCd, statCdCntnKor1 컬럼을 기준으로 expWgt_ratio_2019 컬럼값 transpose 즉, 전치시키는 코드를 제공해주세요.

# Remove rows with missing values in 'expWgt_ratio_2019'
cleaned_df = merged_df.dropna(subset=['expWgt_ratio_2019']).copy()

# Pivot the DataFrame
pivoted_df = cleaned_df.pivot(index=['statCd', 'statCdCntnKor1'], columns='year', values='expWgt_ratio_2019')

# Display the pivoted DataFrame
display(pivoted_df)

,year,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
statCd,statCdCntnKor1,,,,,,,,,,
AE,아랍에미리트 연합,0.297063,0.514908,0.885231,0.842657,1.0,1.292011,3.070706,4.669419,3.848955,5.264214
AF,아프카니스탄,5.0,<NA>,<NA>,<NA>,1.0,<NA>,1090.0,2768.75,3805.0,2812.0
AL,알바니아,<NA>,<NA>,0.002911,1.36754,1.0,0.557496,0.038574,0.040757,0.138282,0.077875
AO,앙골라,311.683824,4.227941,6.025735,0.602941,1.0,6.003676,0.029412,0.845588,0.051471,<NA>
AR,아르헨티나,0.803359,0.859201,1.133428,0.714777,1.0,0.440939,0.977887,1.280813,1.314926,1.635435
...,...,...,...,...,...,...,...,...,...,...,...
WF,왈라스 & 퓨투나 군도,<NA>,<NA>,<NA>,<NA>,1.0,<NA>,<NA>,<NA>,<NA>,<NA>
WS,사모아,0.550369,0.582683,0.631071,0.746453,1.0,0.825097,1.054338,0.83599,1.20434,0.961856
ZA,남아프리카공화국,0.544605,0.327425,0.499203,0.714679,1.0,2.494653,6.417668,4.32457,8.812544,9.695476
